In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from imutils import paths
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os

In [13]:
datasetPath='..\kvasir-dataset'
categories=[
    'dyed-lifted-polyps', 
    'dyed-resection-margins',
    'esophagitis',
    'normal-cecum',
    'normal-pylorus',
    'normal-z-line',
    'polyps',
    'ulcerative-colitis'
]
EPOCHS = 200
Learing_Rate = 1e-3
Batch_Size = 128

In [3]:
def buildModel(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    if backend.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
    # CONV => RELU => POOL layers
    model.add(Conv2D(16, (5, 5), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL layers
    model.add(Conv2D(32, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL layers
    model.add(Conv2D(64, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL layers
    model.add(Conv2D(128, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL layers
    model.add(Conv2D(256, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # FC => RELU layers
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    return model

In [4]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    if iteration == total: 
        print()

In [5]:
data = []
labels = []

print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(datasetPath)))
random.seed(42)
random.shuffle(imagePaths)

printProgressBar(0, len(imagePaths), prefix = 'Progress:', suffix = 'Complete', length = 50)
for i,imagePath in enumerate(imagePaths):
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (96, 96))
    image = img_to_array(image)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    label = categories.index(label)
    labels.append(label)
    printProgressBar(i + 1, len(imagePaths), prefix = 'Progress:', suffix = 'Complete', length = 50)

print("[INFO] loading images completed...")

[INFO] loading images...
Progress: |██████████████████████████████████████████████████| 100.0% Complete
[INFO] loading images completed...


In [6]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)

trainY = to_categorical(trainY, num_classes=8)
testY = to_categorical(testY, num_classes=8)

In [7]:
print ("number of training examples = " + str(trainX.shape[0]))
print ("number of test examples = " + str(testX.shape[0]))
print ("X_train shape: " + str(trainX.shape))
print ("Y_train shape: " + str(trainY.shape))
print ("X_test shape: " + str(testX.shape))
print ("Y_test shape: " + str(testY.shape))

number of training examples = 3000
number of test examples = 1000
X_train shape: (3000, 96, 96, 3)
Y_train shape: (3000, 8)
X_test shape: (1000, 96, 96, 3)
Y_test shape: (1000, 8)


In [8]:
generator = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [9]:
print("[INFO] compiling model...")
strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
model = buildModel(width=96, height=96, depth=3, classes=8)
model.summary()

[INFO] compiling model...
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 16)        1216      
_________________________________________________________________
activation (Activation)      (None, 96, 96, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        12832     
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0    

In [10]:
opt = Adam(lr=Learing_Rate, decay=Learing_Rate / EPOCHS)
model.compile(
    loss="binary_crossentropy", 
    optimizer=opt, 
    metrics=[
        "accuracy",
        "categorical_accuracy",
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.SpecificityAtSensitivity(0.5,name="specificity")
    ]
)

In [14]:
history = model.fit(
    x=generator.flow(trainX, trainY, batch_size=Batch_Size),
    validation_data=(testX, testY), 
    steps_per_epoch=len(trainX) // Batch_Size,
    epochs=EPOCHS, 
    verbose=1)
print("[INFO] training network completed...")

Epoch 1/200
23/23 [==============================] - 44s 2s/step - loss: 0.2428 - accuracy: 0.4718 - categorical_accuracy: 0.4718 - precision: 0.5436 - recall: 0.2256 - specificity: 0.9252 - val_loss: 0.2230 - val_accuracy: 0.5110 - val_categorical_accuracy: 0.5110 - val_precision: 0.5810 - val_recall: 0.4090 - val_specificity: 0.9399
Epoch 2/200
23/23 [==============================] - 43s 2s/step - loss: 0.2214 - accuracy: 0.5279 - categorical_accuracy: 0.5279 - precision: 0.5991 - recall: 0.3642 - specificity: 0.9426 - val_loss: 0.1958 - val_accuracy: 0.5920 - val_categorical_accuracy: 0.5920 - val_precision: 0.6262 - val_recall: 0.5160 - val_specificity: 0.9569
Epoch 3/200
23/23 [==============================] - 43s 2s/step - loss: 0.2041 - accuracy: 0.5557 - categorical_accuracy: 0.5557 - precision: 0.6120 - recall: 0.4499 - specificity: 0.9502 - val_loss: 0.2214 - val_accuracy: 0.5180 - val_categorical_accuracy: 0.5180 - val_precision: 0.5661 - val_recall: 0.4240 - val_specifici

23/23 [==============================] - 40s 2s/step - loss: 0.1286 - accuracy: 0.7451 - categorical_accuracy: 0.7451 - precision: 0.7603 - recall: 0.7221 - specificity: 0.9892 - val_loss: 0.1536 - val_accuracy: 0.7210 - val_categorical_accuracy: 0.7210 - val_precision: 0.7366 - val_recall: 0.6990 - val_specificity: 0.9847
Epoch 26/200
23/23 [==============================] - 40s 2s/step - loss: 0.1329 - accuracy: 0.7403 - categorical_accuracy: 0.7403 - precision: 0.7539 - recall: 0.7169 - specificity: 0.9877 - val_loss: 0.1405 - val_accuracy: 0.7230 - val_categorical_accuracy: 0.7230 - val_precision: 0.7328 - val_recall: 0.7130 - val_specificity: 0.9883
Epoch 27/200
23/23 [==============================] - 41s 2s/step - loss: 0.1250 - accuracy: 0.7597 - categorical_accuracy: 0.7597 - precision: 0.7716 - recall: 0.7389 - specificity: 0.9907 - val_loss: 0.1650 - val_accuracy: 0.6820 - val_categorical_accuracy: 0.6820 - val_precision: 0.6981 - val_recall: 0.6520 - val_specificity: 0.9776

23/23 [==============================] - 44s 2s/step - loss: 0.0914 - accuracy: 0.8437 - categorical_accuracy: 0.8437 - precision: 0.8511 - recall: 0.8381 - specificity: 0.9970 - val_loss: 0.1589 - val_accuracy: 0.7410 - val_categorical_accuracy: 0.7410 - val_precision: 0.7474 - val_recall: 0.7310 - val_specificity: 0.9876
Epoch 50/200
23/23 [==============================] - 48s 2s/step - loss: 0.0943 - accuracy: 0.8304 - categorical_accuracy: 0.8304 - precision: 0.8342 - recall: 0.8196 - specificity: 0.9969 - val_loss: 0.1473 - val_accuracy: 0.7550 - val_categorical_accuracy: 0.7550 - val_precision: 0.7643 - val_recall: 0.7460 - val_specificity: 0.9893
Epoch 51/200
23/23 [==============================] - 51s 2s/step - loss: 0.0880 - accuracy: 0.8430 - categorical_accuracy: 0.8430 - precision: 0.8479 - recall: 0.8350 - specificity: 0.9973 - val_loss: 0.1407 - val_accuracy: 0.7650 - val_categorical_accuracy: 0.7650 - val_precision: 0.7685 - val_recall: 0.7570 - val_specificity: 0.9916

23/23 [==============================] - 41s 2s/step - loss: 0.0704 - accuracy: 0.8799 - categorical_accuracy: 0.8799 - precision: 0.8835 - recall: 0.8764 - specificity: 0.9991 - val_loss: 0.1600 - val_accuracy: 0.7620 - val_categorical_accuracy: 0.7620 - val_precision: 0.7667 - val_recall: 0.7590 - val_specificity: 0.9880
Epoch 74/200
23/23 [==============================] - 41s 2s/step - loss: 0.0740 - accuracy: 0.8719 - categorical_accuracy: 0.8719 - precision: 0.8796 - recall: 0.8646 - specificity: 0.9989 - val_loss: 0.1715 - val_accuracy: 0.7370 - val_categorical_accuracy: 0.7370 - val_precision: 0.7505 - val_recall: 0.7310 - val_specificity: 0.9866
Epoch 75/200
23/23 [==============================] - 42s 2s/step - loss: 0.0733 - accuracy: 0.8788 - categorical_accuracy: 0.8788 - precision: 0.8862 - recall: 0.8733 - specificity: 0.9986 - val_loss: 0.1609 - val_accuracy: 0.7700 - val_categorical_accuracy: 0.7700 - val_precision: 0.7739 - val_recall: 0.7700 - val_specificity: 0.9901

23/23 [==============================] - 42s 2s/step - loss: 0.0531 - accuracy: 0.9074 - categorical_accuracy: 0.9074 - precision: 0.9110 - recall: 0.9053 - specificity: 0.9997 - val_loss: 0.1850 - val_accuracy: 0.7650 - val_categorical_accuracy: 0.7650 - val_precision: 0.7669 - val_recall: 0.7600 - val_specificity: 0.9907
Epoch 98/200
23/23 [==============================] - 42s 2s/step - loss: 0.0492 - accuracy: 0.9164 - categorical_accuracy: 0.9164 - precision: 0.9182 - recall: 0.9143 - specificity: 0.9999 - val_loss: 0.1844 - val_accuracy: 0.7610 - val_categorical_accuracy: 0.7610 - val_precision: 0.7631 - val_recall: 0.7600 - val_specificity: 0.9896
Epoch 99/200
23/23 [==============================] - 42s 2s/step - loss: 0.0417 - accuracy: 0.9300 - categorical_accuracy: 0.9300 - precision: 0.9321 - recall: 0.9276 - specificity: 0.9999 - val_loss: 0.2005 - val_accuracy: 0.7820 - val_categorical_accuracy: 0.7820 - val_precision: 0.7841 - val_recall: 0.7810 - val_specificity: 0.9931

KeyboardInterrupt: 

In [ ]:
print("[INFO] saving model...")
model.save('Model_CNN6.model', save_format="h5")
print("[INFO] model saved")

In [ ]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch No")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
plt.savefig('plot_CNN6.png')

In [ ]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["precision"], label="train_precision")
plt.plot(np.arange(0, N), history.history["recall"], label="train_recall")
plt.plot(np.arange(0, N), history.history["specificity"], label="train_specificity")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), history.history["val_precision"], label="val_precision")
plt.plot(np.arange(0, N), history.history["val_recall"], label="val_recall")
plt.plot(np.arange(0, N), history.history["val_specificity"], label="val_specificity")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_accuracy")
plt.title("Training Metrics")
plt.xlabel("Epoch No")
plt.ylabel("Value")
plt.legend(loc="lower left")
plt.show()
plt.savefig('plot_CNN6_metrics.png')